In [0]:
import numpy as np
import matplotlib.pyplot as plt
import keras as keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D , Dropout,Conv3D, MaxPooling3D, Input, Deconv3D, BatchNormalization, Activation, Reshape
from keras.utils import to_categorical
from keras import optimizers

In [100]:
filters_in = [96, 256, 384,384,256]
kernel_size = [4,8,16,16,8]
strides = (2,2,2)
def encoder2D(kernel_size, strides):
    inputs = Input(shape=(512,1024,1))

    g1 = Conv2D(filters=filters_in[0], kernel_size=kernel_size[0])(inputs)
    g1 = MaxPooling2D(pool_size=2)(g1)
    g1 = Activation(activation='relu')(g1)

    g2 = Conv2D(filters=filters_in[1], kernel_size=kernel_size[1])(g1)
    g2 = MaxPooling2D(pool_size=2)(g2)
    g2 = Activation(activation='relu')(g2)

    g3 = Conv2D(filters=filters_in[2], kernel_size=kernel_size[2])(g2)
    g3 = MaxPooling2D(pool_size=2)(g3)
    g3 = Activation(activation='relu')(g3)

    g4 = Conv2D(filters=filters_in[3], kernel_size=kernel_size[3])(g3)
    g4 = MaxPooling2D(pool_size=2)(g4)
    g4 = Activation(activation='relu')(g4)

    g5 = Conv2D(filters=filters_in[4], kernel_size=kernel_size[4])(g4)
    g5 = MaxPooling2D(pool_size=2)(g5)
    g5 = Activation(activation='sigmoid')(g5) 
    
    g6 = Flatten()(g5)
    
    g7 = Dense(400,activation='sigmoid')(g6)
    g7 = Activation(activation='sigmoid')(g7) 
    
    g8 = Dense(200,activation='sigmoid')(g7)
    g8 = Activation(activation='sigmoid')(g8) 
    
    g9 = Reshape((1, 1,1,200), input_shape=(200,))(g8)
    
    
    model = Model(inputs=inputs, outputs=g9)
    model.summary()

    return model 
encoder2D = encoder2D(kernel_size,strides)  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_68 (InputLayer)        (None, 512, 1024, 1)      0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 509, 1021, 96)     1632      
_________________________________________________________________
max_pooling2d_96 (MaxPooling (None, 254, 510, 96)      0         
_________________________________________________________________
activation_262 (Activation)  (None, 254, 510, 96)      0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 247, 503, 256)     1573120   
_________________________________________________________________
max_pooling2d_97 (MaxPooling (None, 123, 251, 256)     0         
_________________________________________________________________
activation_263 (Activation)  (None, 123, 251, 256)     0         
__________

In [101]:
kernel_size = [4,8,32,100,100]
strides = (2,2,2)
def generator(kernel_size, strides):
    inputs = Input(shape=(1, 1, 1, 200))

    g1 = Deconv3D(filters=512, kernel_size=kernel_size[0],strides=strides)(inputs)
    g1 = BatchNormalization()(g1)
    g1 = Activation(activation='relu')(g1)

    g2 = Deconv3D(filters=256, kernel_size=kernel_size[1],strides=strides)(g1)
    g2 = BatchNormalization()(g2)
    g2 = Activation(activation='relu')(g2)

    g3 = Deconv3D(filters=128, kernel_size=kernel_size[2],strides=strides)(g2)
    g3 = BatchNormalization()(g3)
    g3 = Activation(activation='relu')(g3)

    g4 = Deconv3D(filters=64, kernel_size=kernel_size[3])(g3)
    g4 = BatchNormalization()(g4)
    g4 = Activation(activation='relu')(g4)

    g5 = Deconv3D(filters=1, kernel_size=kernel_size[4])(g4)
    g5 = BatchNormalization()(g5)
    g5 = Activation(activation='sigmoid')(g5) 

    model = Model(inputs=inputs, outputs=g5)
    model.summary()

    return model 
generator = generator(kernel_size,strides)  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_69 (InputLayer)        (None, 1, 1, 1, 200)      0         
_________________________________________________________________
conv3d_transpose_234 (Conv3D (None, 4, 4, 4, 512)      6554112   
_________________________________________________________________
batch_normalization_232 (Bat (None, 4, 4, 4, 512)      2048      
_________________________________________________________________
activation_269 (Activation)  (None, 4, 4, 4, 512)      0         
_________________________________________________________________
conv3d_transpose_235 (Conv3D (None, 14, 14, 14, 256)   67109120  
_________________________________________________________________
batch_normalization_233 (Bat (None, 14, 14, 14, 256)   1024      
_________________________________________________________________
activation_270 (Activation)  (None, 14, 14, 14, 256)   0         
__________

In [105]:
def create_model():
  model = Sequential()
  model.add(encoder2D)
  model.add(generator)
  model.summary()
  return model

model = create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_47 (Model)             (None, 1, 1, 1, 200)      84379192  
_________________________________________________________________
model_48 (Model)             (None, 256, 256, 256, 1)  9403409093
Total params: 9,487,788,285
Trainable params: 9,487,786,363
Non-trainable params: 1,922
_________________________________________________________________
